In [1]:
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date, ForeignKey, Sequence, CheckConstraint, UniqueConstraint

In [2]:
Base = declarative_base()

C:\Users\domin\AppData\Local\Temp\ipykernel_3916\4196137762.py:1: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
# create tables structure
class TextContent(Base):
    __tablename__ = 'TextContent'
    __table_args__ = (
        UniqueConstraint('text_content_tag'),
    )
    text_content_id = Column(Integer, primary_key=True)
    text_content_tag = Column(String(50), nullable=False)
    original_text = Column(String(500))
    original_language_id = Column(Integer, ForeignKey('Languages.language_id'))

    def __repr__(self):
        return "<TextContent(text_content_id={0}, text_content_tag={1}, original_text={2}, original_language_id={3})>".format(
            self.text_content_id, self.text_content_tag, self.original_text, self.original_language_id)
    
class Languages(Base):
    __tablename__ = 'Languages'
    __table_args__ = (
        CheckConstraint('LENGTH(language_name) = 2'),
        UniqueConstraint('language_name'),
    )
    language_id = Column(Integer, primary_key=True)
    language_name = Column(String(50), nullable = False)

    def __repr__(self):
        return "<Languages(language_id={0}, language_name={1})>".format(
            self.language_id, self.language_name)
    
class Translations(Base):
    __tablename__ = 'Translations'
    translation_id = Column(Integer, primary_key=True)
    text_content_id = Column(Integer, ForeignKey("TextContent.text_content_id"))
    language_id = Column(Integer, ForeignKey("Languages.language_id"))
    translation = Column(String(500))

    def __repr__(self):
        return "<Translations(translation_id={0}, text_content_id={1}, language_id={2}, translation={3})>".format(
            self.translation_id, self.text_content_id, self.language_id, self.translation)

In [4]:
# connect to database
engine = create_engine("postgresql://postgres@localhost/millionaires")

# create database if not exists
if not database_exists(engine.url):
    create_database(engine.url)
    Base.metadata.create_all(engine)

db = engine.connect()

In [5]:
from sqlalchemy import MetaData, Table, insert

# create tables using ORM techniques
metadata = MetaData()
languages = Table('Languages', metadata, autoload_with=db)
text_content = Table('TextContent', metadata, autoload_with=db)

# insert original language
query_languages = insert(languages).values(language_id=1, language_name='pl')
# insert original text content
query_text_content = insert(text_content).values(
[
    {'text_content_id': 1, 'text_content_tag': '50_50', 'original_text': '50 / 50', 'original_language_id': 1}
]
)
# execute queries
result_languages = db.execute(query_languages)
result_text_content = db.execute(query_text_content)
db.commit()